In [16]:


import pandas as pd
import io
import requests
url="https://s3.amazonaws.com/hr-testcases/859/assets/training.json"
s=requests.get(url).content
train=pd.read_json(io.StringIO(s.decode('utf-8')), lines=True)
train=train.loc[1:,:]

#for col in train.columns: 
#    print(col) 

#print (train[0])
train['city']=train.iloc[:,0].apply(lambda x:x['city'])
train['category']=train.iloc[:,0].apply(lambda x:x['category'])
train['section']=train.iloc[:,0].apply(lambda x:x['section'])
train['heading']=train.iloc[:,0].apply(lambda x:x['heading'])
train=train.iloc[:,1:]
print (train.head())
# print (train.shape)

#we are supossed to predict the category given the city, section and heading
train.reset_index(inplace=True,drop=True)
print (train.head())

#let's start the modelling process
#1st we will be treating the categorical variables, since there is no
#order in these so we will be using 1 hot encoding
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# le.fit(train['city'])
# train['city_le'] = le.transform(train['city'])

# le = preprocessing.LabelEncoder()
# le.fit(train['section'])
# train['section_le'] = le.transform(train['section'])


city_one_hot=pd.get_dummies(train['city'], prefix='city')
train = pd.concat([train,city_one_hot],axis=1)

section_one_hot=pd.get_dummies(train['section'], prefix='section')
train = pd.concat([train,section_one_hot],axis=1)
# print (train.head())
# print (train.shape)

import re
#Now doing tfidf vectorization with stop word and stemming and removal of numbers
train['heading']=train['heading'].str.lower()
train['heading']=train['heading'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
train['heading']=train['heading'].apply(lambda x:re.sub(r'\W', ' ', str(x)))
train['heading']=train['heading'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english',max_features=1000,ngram_range=(1,2),sublinear_tf=True)
X = vectorizer.fit_transform(train['heading'])


X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())


train = pd.concat([train,X_df], axis=1)

train.drop( ['city','section','heading'], axis=1,inplace=True)

#Now using the naive bayes algorithm to do the prediction
X =train.loc[:, train.columns != 'category']
y = train['category']
#print (X.head())
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
          
gnb = GaussianNB()
gnb.fit(X_train, y_train)

print (gnb.score(X_test,y_test))

#print (gnb.predict(X_test))




#Now using the model to predict on test
url="https://s3.amazonaws.com/hr-testcases/859/sample-test.in.json"
s=requests.get(url).content
test=pd.read_json(io.StringIO(s.decode('utf-8')), lines=True)
test=test.loc[1:,:]

#for col in train.columns: 
#    print(col) 

#print (train[0])
test['city']=test.iloc[:,0].apply(lambda x:x['city'])
test['section']=test.iloc[:,0].apply(lambda x:x['section'])
test['heading']=test.iloc[:,0].apply(lambda x:x['heading'])
test=test.iloc[:,1:]
test.reset_index(inplace=True,drop=True)

city_one_hot=pd.get_dummies(test['city'], prefix='city')
test = pd.concat([test,city_one_hot],axis=1)

section_one_hot=pd.get_dummies(test['section'], prefix='section')
test = pd.concat([test,section_one_hot],axis=1)
# print (train.head())
# print (train.shape)

#Now doing tfidf vectorization with stop word and stemming and removal of numbers
test['heading']=test['heading'].str.lower()
test['heading']=test['heading'].apply(lambda x:re.sub(r'\s+', ' ', x, flags=re.I))
test['heading']=test['heading'].apply(lambda x:re.sub(r'\W', ' ', str(x)))
test['heading']=test['heading'].apply(lambda x:re.sub(r'\s+[a-zA-Z]\s+', ' ', x))

X = vectorizer.transform(test['heading'])  #dont use fit_transform here, as we are using the 
#vectorizer we used for training and just fitting the test data to it


X_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())


test = pd.concat([test,X_df], axis=1)
test.drop( ['city','section','heading'], axis=1,inplace=True)


url="https://s3.amazonaws.com/hr-testcases/859/sample-test.in.json"
s=requests.get(url).content
test1=pd.read_json(io.StringIO(s.decode('utf-8')), lines=True)
test1=test1.loc[1:,:]

#for col in train.columns: 
#    print(col) 

#print (train[0])
test1['city']=test1.iloc[:,0].apply(lambda x:x['city'])
test1['section']=test1.iloc[:,0].apply(lambda x:x['section'])
test1['heading']=test1.iloc[:,0].apply(lambda x:x['heading'])
test1=test1.iloc[:,1:]
test1.reset_index(inplace=True,drop=True)


x=0
for i in gnb.predict(test):
    print (test1.loc[x,'heading'])
    print ('category '+str(i))
    x=x+1


       



      city     category   section  \
1  newyork  cell-phones  for-sale   
2  newyork  cell-phones  for-sale   
3  newyork  cell-phones  for-sale   
4  newyork  cell-phones  for-sale   
5  newyork  cell-phones  for-sale   

                                             heading  
1  New batteries C-S2 for Blackberry 7100/7130/87...  
2  *******   Brand New Original SAMSUNG GALAXY NO...  
3  SAMSUNG GALAXY SIII T-999 MARBLE WHITE T-MOBIL...  
4                  Ipad mini 64gb 4g any sim unlock   
5                           htc evo 4g lte for trade  
      city     category   section  \
0  newyork  cell-phones  for-sale   
1  newyork  cell-phones  for-sale   
2  newyork  cell-phones  for-sale   
3  newyork  cell-phones  for-sale   
4  newyork  cell-phones  for-sale   

                                             heading  
0  New batteries C-S2 for Blackberry 7100/7130/87...  
1  *******   Brand New Original SAMSUNG GALAXY NO...  
2  SAMSUNG GALAXY SIII T-999 MARBLE WHITE T-MOBIL...  
3   

category housing
map
category video-games
Bedbug Specialist Open 7 Days a Week
category real-estate
FIRST MONTH FREE 
category cell-phones
2nd floor gorgeous 2 bedroom 
category temporary
Books And AV Materials Needed
category activities
Government Support for 60yo and older
category activities
WINDOW TINTING -$100
category real-estate
big double room to rent in a house share. All bills included. 3 levels
category shared
map
category temporary
white electric stove/range good working condition
category appliances
iPhone 5 64GB Black Unlocked
category cell-phones
 Yes! You too can become successful at getting women
category therapeutic
needed 1 bedroom apartment for 2 
category temporary
map
category temporary
49th West Luxury Condo Doorman Fully Studio/Sept 15 
category temporary
***GOOD Xbox 360 Games***
category video-games
map
category housing
 &#10152;&#10152;&#10152;Move in/Move out cleaning!!! 
category real-estate
Babysitter &amp; Nanny
category childcare
New stainless steel frid

category video-games
Studio furnished apartment in Upper East Side, NY-15831
category temporary
We will buy your property!!
category real-estate
$700 Upper east coast road , aircon room for rent for single guy lady
category shared
********* This Is Where The Money Is *********
category artists
map
category housing
nexus 7 like new
category cell-phones
Friendly student female looking for inexpensive room in zone 1
category wanted-housing
WATER COOLER
category appliances
MODDED XBOX 360 WITH GAMES
category video-games
SEPT 15*SPACIOUS*GORGEOUS*SUNNY*OPEN VIEWS!*BALCONY*24HR DM*H/C*PICS!
category temporary
NEWSTEPS CONCRETE
category activities
Furnished Room in Beautiful House
category temporary
cheap hookah cigars
category video-games
*** JOIN THE ENTREPRENEURIAL REVOLUTION!! ***
category automotive
Contax G1 (green label) with zeiss planar 35mm f2
category photography
male to male body massge by model in delhi prince 09953551666
category therapeutic
broken/working pink ds lite
category 

map
category video-games
Mom w/kids!!
category wanted-housing
man only massage 0554699774 - aed 200 (Hourly)
category therapeutic
New IPhone 5 
category cell-phones
 	Fist Time In Hyderabad Coming Call Mohit Fun
category therapeutic
©&#8902;&#8902;&#8902;© NEW  6.00 Place Settings Counter-top Dishwasher
category appliances
Disc Replay Repair Service XBOX PS3 Wii PSP DS IPod Touch Classic 
category video-games
Offer long and short term booking
category temporary
Room in a bed and breakfast
category temporary
Mobile Auto Repair Save a Tow 
category automotive
My Downtown LA for your Paris
category temporary
Cash Offer For Your House
category real-estate
Softball Tournament &amp; BBQ Sept 1st 
category childcare
map
category temporary
map
category automotive
**********EXPERIENCED PAINTER AND HANDYMAN**********
category real-estate
Large bedroom in a sunny 1200 sqft apt Canal St Martin
category temporary
map
category automotive
Babysitting job with LIBERT'HOME
category childcare
Babysitter

CLEANING...CLEANING. FIRST TIME CLEANING SPECIAL RATES. 
category real-estate
Want to buy or sell a house? Call me!
category real-estate
Bolex M8 Projector
category video-games
Park Avenue
category temporary
Treo Verizon &amp; Motorola Q-Used Page Plus Monthy $12 250 Mins/250text 
category cell-phones
map
category automotive
Looking for a RESPONSIBLE EMT
category childcare
FEMALE ROOMMATE WANTED.....ALL UTILITIES INCLUDED.....................
category shared
map
category activities
After School Babysitter 3-5 days per week
category childcare
Minolta AF APO High Speed 600mm f4 Tele 600F4
category photography
I Buy Luxury House
category real-estate
PG ROOM AVAILABLE AT WORLI CALL MANISH - 9820852868
category shared
Got room for one more on your Sailboat?
category artists
 &#9626; Emergency Cleaning*Today &amp; Tomorrow
category real-estate
Single Muslim Girls
category general
Autotronics NW GM-Based cluster repair
category automotive
map
category temporary
Billingham khaki canvas/tan lea

...

category temporary
EXPERIENCED PLUMBER~~~EMANUEL~~~773-664-1693~~~
category household-services
In need of a SAHM or affordable childcare 
category childcare
CAMERA COLLECTORS!!! 3 Minox Cameras
category photography
2 pièces 46m2 loc semaine(week) ou mois (month)
category temporary
Fujifilm Finepix F10 Digital Camera Great condition with Card, Batte
category photography
Wrigleyville 3BR 1.5BA Updated
category wanted-housing
Hyderabad call girls Mr.Atul- 9701068787 -Call girls in hyderabad
category therapeutic
Stop Foreclosure/Auction
category real-estate
map
category video-games
map
category activities
Grand Theft Auto for PC
category photography
GREAT HOUSING OPTION FOR STUDENTS! FULLY-FURNISHED APTS! NO LEASE!
category temporary
ATTENTION: FOOTBALL PLAYERS WANTED!!!!!!
category activities
map
category housing
Live-In Nanny Available (speaks Italian, Hindi, Punjabi)
category childcare
Flats and Residential Property in Rohini
category real-estate
Large single room
category shared
GREAT 

map
category video-games
Sigma 85mm f/1.4 EX for Canon like new!!
category photography
map
category housing
IM A HASA CLIENT LOOKING FOR A STUDIO APT
category wanted-housing
babbysitter in need of a job asap 
category childcare
Aug-Dec in STYLISH APT 10min fr St G des Pres- wifi,kitchen,wash&amp;dryer
category temporary
EX Modified Contax G Carl Zeiss 21,28,45,90mm Lens Set For PL Mount Ar
category photography
BRAND NEW TWO BEDROOM APARTMENT
category shared
Near Clementi Rd/Holland Rd/Orchard, NO OWNER Nice Furnish Aircon Room
category shared
$25 A Night Crash Pad JFK Airline Only
category temporary
Please Help!  Need Sublet, Temp, or Apt. ASAP
category wanted-housing
2x Fuji NP-70/Ricoh DB-60/Panasonic DMW-BCC12 batteries + charger
category photography
&#9792; Nuru slippery massage with everything - dream come true &#9794;
category therapeutic
Looking for room in Cobbler Square Lofts
category wanted-housing
 &#9792; &#10084;B2B&#10084;Luscious sensual massage with mutual touching&#979

LINCOLN PK. &amp; GOLDCOAST JUNK CAR REMOVAL SERVICE BUYS JUNK CARS 
category automotive
Panasonic,2 door fridge (395L)($350 with delivery+1mth warranty)
category appliances
Last minute! Promo/W , heart of Paris, 2 arrdt ,wifi ,renovated
category temporary
NEXTEL &amp; Sprint PHONES
category cell-phones
GE ELECTRIC STOVE 220 LINE
category appliances
PAINTING?Highly Skilled, Pro Painter-Interior/Exter&amp;Wallpaper
category real-estate
 &#9807; Gorgeous bu.sty Model and masseuse welcomes you today..!!
category therapeutic
Pelican 1600 camera case for Canon or Nikon
category photography
Instant cash for cars and trucks
category automotive
Unlock Your Spint/Verizon iPhone 4s 
category cell-phones
Xbox 360 elite 250 OBO!
category video-games
 Looking for a nice female friend
category activities
Lighting &amp; Grips,Bulbs, etc 
category video-games
map
category video-games
lg venon mother board
category cell-phones
MOBILE IS NOW MONEY MAKING MACHINE !!!!
category cell-phones
  * * * EVICTIO

Kenmore dryer
category appliances
FREE Bodywrap!
category activities
Nikon 18-105 vr lens
category photography
map
category video-games
One bedroom at Hertsmere Road, Docklands
category temporary
no registration fee
category childcare
First Friends Licensed Daycare and Preschool in fun home setting 
category childcare
&#10084;&#10084;SWEETA==========ESCORTS==========SERVICE&#10084;&#10084;
category therapeutic
Call for Submissions - Writers
category activities
Panasonic Micro 4/3 GX1 With 14-42mm Zoom Lense -- New for SGD 999
category photography
Du&#1089;tless Air  onditioner Systems==
category appliances
TAG SALE
category general
No Owner! Master rm, Moro Mansion, Paya Lebar MRT, $1300
category temporary
Independent Modeles Airhosteres Acting Modeles And Danceres Real 
category therapeutic
 XBOX 360 Hard Drive Transfer Disc
category video-games
great charming 1br/1bath . great building ** prime location ** W3  
category housing
PROFESSIONAL CLEANERS! BEST RATES! 10% OFF NEW CUSTOMERS

&#9820;i buy "BROKEN" iPhones CASH!
category cell-phones
Canon 5D Mark III Body Only
category photography
map
category activities
kitchenaid stand mixer, model # ksm90-excellent condition
category photography
French Nanny available
category childcare
Mobile Auto Repair---Just Like The Shop But Cheaper
category automotive
Ikea painting 
category video-games
Hyderabad Call &#8458;irls Mr.Atul-9701068787- Call  &#8458;irls  In  Hyderabad
category therapeutic
 FOR SALE: www.ChicagoPropertyMaintenance.com (web domain only)
category real-estate
sony nex SEL16F28 16mm f/2.8 Wide Angle Lens 
category photography
Brand new Chi flat iron for straightening hair
category appliances
map
category activities
PS3 White 500GB
category video-games
COUSIN'S LABOR SERVICES
category automotive
Doble room for share wanted inexpensive 400
category wanted-housing
  man only massage 0554699774 - aed 200 (Hourly)
category therapeutic
LOOKING FOR A CHILL FEMALE HOUSEMATE
category wanted-housing
&#920; Seize the 